In [16]:
import os
import glob
import h5py
import numpy as np
from keras import backend as K
from keras.models import Sequential, load_model
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten, Dense, Activation, Reshape
from keras.layers import Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

In [2]:
print(K.image_dim_ordering())
img_width, img_height = 150, 150
root_train_folder_path = os.path.join('database', 'train')
train_data_dir = os.path.join(root_train_folder_path, 'train')
validation_data_dir = os.path.join(root_train_folder_path, 'validation')

tf


In [3]:
# used to rescale the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255)

In [4]:
# automagically retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=16,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=4,
        class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [5]:
nb_train_samples = 20000
nb_validation_samples = 5000

In [6]:
model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=(img_width, img_height, 3),padding='same', activation='relu', name='conv1_1'))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu', name='conv1_2'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), padding='same', activation='relu', name='conv2_1'))
model.add(Conv2D(128, (3, 3), padding='same', activation='relu', name='conv2_2'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), padding='same', activation='relu', name='conv3_1'))
model.add(Conv2D(256, (3, 3), padding='same', activation='relu', name='conv3_2'))
model.add(Conv2D(256, (3, 3), padding='same', activation='relu', name='conv3_3'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(512, (3, 3), padding='same', activation='relu', name='conv4_1'))
model.add(Conv2D(512, (3, 3), padding='same', activation='relu', name='conv4_2'))
model.add(Conv2D(512, (3, 3), padding='same', activation='relu', name='conv4_3'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(ZeroPadding2D((1, 1)))
# model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
# model.add(ZeroPadding2D((1, 1)))
# model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
# model.add(ZeroPadding2D((1, 1)))
# model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
# model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu', name='dense1'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1_1 (Conv2D)             (None, 150, 150, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 150, 150, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 64)        0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 75, 75, 128)       73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 75, 75, 128)       147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 37, 37, 128)       0         
_________________________________________________________________
conv3_1 (Conv2D)             (None, 37, 37, 256)       295168    
__________

In [8]:
model.compile(loss='binary_crossentropy',
          optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
          metrics=['accuracy'])

In [14]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=800, 
        verbose=1)

Epoch 1/10
2000/2000 [==============================] - 351s - loss: 0.6914 - acc: 0.5461 - val_loss: 0.6905 - val_acc: 0.5225
Epoch 2/10
2000/2000 [==============================] - 351s - loss: 0.6897 - acc: 0.5574 - val_loss: 0.6880 - val_acc: 0.5991
Epoch 3/10
2000/2000 [==============================] - 350s - loss: 0.6852 - acc: 0.5809 - val_loss: 0.6806 - val_acc: 0.5997
Epoch 4/10
2000/2000 [==============================] - 349s - loss: 0.6723 - acc: 0.5895 - val_loss: 0.6642 - val_acc: 0.5809
Epoch 5/10
2000/2000 [==============================] - 350s - loss: 0.6577 - acc: 0.6027 - val_loss: 0.6465 - val_acc: 0.6122
Epoch 6/10
2000/2000 [==============================] - 350s - loss: 0.6501 - acc: 0.6073 - val_loss: 0.6859 - val_acc: 0.5487
Epoch 7/10
2000/2000 [==============================] - 351s - loss: 0.6422 - acc: 0.6189 - val_loss: 0.6300 - val_acc: 0.6191
Epoch 8/10
2000/2000 [==============================] - 350s - loss: 0.6328 - acc: 0.6276 - val_loss: 0.6211 - 

In [15]:
models_folder_path = os.path.join("models")
model_name = "simple_with_dropout"
model.save(os.path.join(models_folder_path, model_name) + '-steps{}.h5'.format(10))

In [18]:
loaded_model = load_model(os.path.join(models_folder_path, model_name) + '-steps{}.h5'.format(10))

In [19]:
loaded_model.fit_generator(
        train_generator,
        steps_per_epoch=2000,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800, 
        verbose=1)

Epoch 1/50
2000/2000 [==============================] - 352s - loss: 0.5983 - acc: 0.6590 - val_loss: 0.5962 - val_acc: 0.6716
Epoch 2/50
2000/2000 [==============================] - 350s - loss: 0.5870 - acc: 0.6739 - val_loss: 0.5796 - val_acc: 0.6847
Epoch 3/50
2000/2000 [==============================] - 351s - loss: 0.5760 - acc: 0.6821 - val_loss: 0.5675 - val_acc: 0.6931
Epoch 4/50
2000/2000 [==============================] - 350s - loss: 0.5613 - acc: 0.7009 - val_loss: 0.5492 - val_acc: 0.7228
Epoch 5/50
2000/2000 [==============================] - 351s - loss: 0.5492 - acc: 0.7110 - val_loss: 0.5314 - val_acc: 0.7366
Epoch 6/50
2000/2000 [==============================] - 351s - loss: 0.5362 - acc: 0.7226 - val_loss: 0.5188 - val_acc: 0.7453
Epoch 7/50
2000/2000 [==============================] - 350s - loss: 0.5285 - acc: 0.7330 - val_loss: 0.5233 - val_acc: 0.7416
Epoch 8/50
2000/2000 [==============================] - 350s - loss: 0.5150 - acc: 0.7420 - val_loss: 0.5041 - 

In [21]:
loaded_model.save(os.path.join(models_folder_path, model_name) + '-steps{}.h5'.format(60))